In [1]:
class fetch:
    def __init__(self, doc):
        self.doc = doc
    
    def ids(self):
        try: return self.doc['clinical_study']['id_info']['nct_id']
        except: return 'N/A'
    def agency(self):
        try : return self.doc['clinical_study']['sponsors']['lead_sponsor']['agency']
        except: return 'N/A'
    def brief_summary(self):
        try: return self.doc['clinical_study']['brief_summary']['textblock']
        except: return 'N/A'
    def overall_status(self):
        try: return self.doc['clinical_study']['overall_status']
        except: return 'N/A'
    def study_type(self):
        try: return self.doc['clinical_study']['study_type']
        except: return 'N/A'
    def condition(self):
        try: return self.doc['clinical_study']['condition']
        except: return 'N/A'
    def gender(self):
        try: return self.doc['clinical_study']['eligibility']['gender']
        except: return 'N/A'

In [2]:
import sqlite3
conn = sqlite3.connect('data.db')
cur = conn.cursor()
cur.executescript('''
DROP TABLE IF EXISTS agency;
DROP TABLE IF EXISTS briefsummary;
DROP TABLE IF EXISTS overallstatus;
DROP TABLE IF EXISTS studytype;
DROP TABLE IF EXISTS gender; ''')
# DROP TABLE IF EXISTS condition;

cur.execute('''CREATE TABLE agency (ids TEXT, agencyname TEXT)''')
cur.execute('''CREATE TABLE briefsummary (ids TEXT, summary TEXT)''')
cur.execute('''CREATE TABLE overallstatus (ids TEXT, status TEXT)''')
cur.execute('''CREATE TABLE studytype (ids TEXT, type TEXT)''')
#cur.execute('''CREATE TABLE condition (ids TEXT, conditionname)''')
cur.execute('''CREATE TABLE gender (ids TEXT, genderdesc TEXT)''')

In [3]:
import os
import fnmatch
files = []
for root, dirnames, filenames in os.walk(os.getcwd()):
    for filename in fnmatch.filter(filenames, '*.xml'):
        files.append(os.path.join(root, filename))

# len(files)

import xmltodict

for file in files:
    with open(file) as fd:
        try:
            doc = xmltodict.parse(fd.read())
        except:
            continue
    new = fetch(doc)
    ids = new.ids()
    cur.execute('''INSERT INTO agency (ids, agencyname) VALUES ( ?, ? )''', (ids, new.agency()) )
    cur.execute('''INSERT INTO briefsummary (ids, summary) VALUES ( ?, ? )''', (ids, new.brief_summary()) )
    cur.execute('''INSERT INTO overallstatus (ids, status) VALUES ( ?, ? )''', (ids, new.overall_status()) )
    cur.execute('''INSERT INTO studytype (ids, type) VALUES ( ?, ? )''', (ids, new.study_type()) )
    #cur.execute('''INSERT INTO condition (ids, conditionname) VALUES ( ?, ? )''', (ids, new.condition()) )
    cur.execute('''INSERT INTO gender (ids, genderdesc) VALUES ( ?, ? )''', (ids, new.gender()) )
try:
    conn.commit()
except:
    pass
cur.close()